# Setup

In [1]:

import os
import openai
import requests
 

client = openai.OpenAI(api_key=input())
os.environ["OPENAI_API_KEY"] = input() 
def llm(prompt, stop=["\n"]):
    response = client.chat.completions.create(
      model="gpt-4-turbo",
      messages=[{'role' : 'user', 'content' : prompt}],
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response.choices[0].message.content

In [2]:
%pip install gym

Note: you may need to restart the kernel to use updated packages.


In [3]:
import scraper
env = scraper.WikiSearch(api_key=input())

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# $\textrm{tea}_{\textrm{LL}}\textrm{m}$

In [4]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

with open('prompts/team.txt', 'r') as f:
    prompt = '\n'.join(f.readlines())
def webthink(question, prompt=prompt, to_print=True):
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        try:
            thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        except Exception as e:
            print(f'too long? bad error {e}')
            return 0, info
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if prompt.count(' ') > 10_000:
            print('TOO LONG TOO LONG')
            return 0, info
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish | NO ANSWER REACHED")

In [5]:
webthink(question='What musical did North West get hate for her performance in?',
         model='teaLLM')

Thought 1: I need to find out which musical North West performed in that led to her receiving criticism. I should start by searching for information about North West's involvement in musical performances.

Action 1: search | North West | What musical did North West perform in? | musical performance
Observation 1: The text provided does not contain any information about North West performing in a musical. It discusses the points of the compass and their use in navigation and cartography.

Thought 2: The search result seems to have returned information about the geographical concept of "North West" rather than the person North West. I need to refine my search to specifically target North West, the daughter of Kim Kardashian and Kanye West, and her involvement in musical performances.

Action 2: search | North West Kim Kardashian | What musical did North West perform in? | musical performance
Observation 2: North West performed "I Just Can't Wait to Be King" at the Hollywood Bowl's The Li